### Поиск врачей-неврологов, владеющих навыками функциональной диагностики в неврологии

Этот небольшой проект призван помочь в решении важной задачи по подбору персонала для медицинской клиники.
Необходимо проанализировать специалистов-неврологов, которые ведут прием в г. Санкт-Петербурге, и выделить врачей, которые владеют навыками функциональной и УЗИ-диагностики.
Для анализа использовалась база данных prodoctorov.

В проекте есть три основных компонента:
1. Веб-скрейпинг, реализованный посредством пакета selenium
2. Извлечение содержимого html-страницы посредством BeautifulSoup.
3. Поиск по ключевым словам в анкете врача с вынесением якорных фраз в отдельный столбец для дальнейшего анализа HR.

In [24]:
import pandas as pd
import numpy as np

import random
from time import sleep

from bs4 import BeautifulSoup

from selenium import webdriver
from tqdm import notebook

### Определение ключевых слов
В финале будут использоваться два параметра - наличие сертификата пройденного обучения по специальностям "функциональная диагностика" или "УЗИ-диагностика", а также конкретные навыки.

In [25]:
FDorUSD = [r'функциональн.+ диагностик.+', ' фд ', 'узи', 'ультразвук', 'ультразвуковая диагностика']
SKILLS = ['ЭНМГ', 'электронейромиограф', 'электромиограф', 'узи нервов', 'узи бца', 'узи брахиоцефальных артертий', 'дуплекс']

### Функции скрейпинга и поиска ключевых слов

Функция get_doctors() отправляет запрос к каждой из 104 страниц списка врачей-неврологов г. Санкт-Петербурга.
В тексте страницы определяются все ссылки, ведущие на профиль врача.
После этого бот переходит по каждой из этих ссылок, собирая информацию в soup-объект.
Этот объект далее парсится по  5 параметрам: ФИО, профили работы, текст "Доктор о себе", список курсов повышения квалификации, и перечень этапов основного образования.

Функция get_doctor_features используется в методе apply на сформированном датасете, и извлекает для каждой строчки ключевые слова в целевых полях.

In [26]:
def get_doctors():
    doctors = []
    for page in notebook.tqdm(range(1,105)):
        url = 'https://prodoctorov.ru/spb/nevrolog/?page=' + str(page)

        driver = webdriver.Safari()
        driver.get(url)
        html = driver.execute_script("return document.documentElement.outerHTML;")

        soup = BeautifulSoup(html, 'html.parser')

        links = soup.find_all('a', {"class": "b-doctor-card__name-link"})
        for l in links:
            href = l.get('href')
            dr_url = 'https://prodoctorov.ru' + href
            driver.get(dr_url)
            dr_html = driver.execute_script("return document.documentElement.outerHTML;")
            dr_soup = BeautifulSoup(dr_html, 'html.parser')

            doctor = {'link': dr_url}

            try:
                doctor['name'] = " ".join(dr_soup.find_all('span',
                                                           {"itemprop": "name", "class": "d-block ui-text ui-text_h5 ui-text_color_black mb-2"})[0].contents[0].split())
            except IndexError:
                pass

            try:
                doctor['about'] = [" ".join(x.contents[0].split()) for x in dr_soup.find_all('div', {"class": "b-doctor-details__paragraph"})]
            except IndexError:
               pass

            try:
                doctor['profile'] = [x.contents[0] for x in dr_soup.find_all('a', {"class": "b-doctor-details__link b-doctor-details__link_column"})]
            except IndexError:
                pass

            try:
                courses = dr_soup.find_all('div', {"id": "courses"})[0]
                years = [x.contents[0] for x in courses.find_all('div', {"class": "b-doctor-details__number"})]
                course_names = [x.contents[0] for x in courses.find_all('div', {"class": "b-doctor-details__list-item-title"})]
                courses_text = [x + ' (' + y + ')' for x, y in zip(course_names, years)]

                doctor['courses'] = courses_text
            except IndexError:
                pass

            try:
                edu = dr_soup.find_all('div', {"id": "educations"})[0]
                edu_years = [x.contents[0] for x in edu.find_all('div', {"class": "b-doctor-details__number"})]
                edu_names = [" ".join(x.contents[0].split()) for x in edu.find_all('div', {"class": "b-doctor-details__list-item-title"})]
                edu_text = [x + ' (' + y + ')' for x, y in zip(edu_names, edu_years)]
                doctor['education'] = edu_text
            except IndexError:
                pass

            doctors.append(doctor)
            time.sleep(random.uniform(0, 2))
        driver.close()

    return(doctors)

def get_doctor_features(row):
    try:
        education = [x.lower() for x in row['education']]
        courses = [x.lower() for x in row['courses']]
        about = [x.lower() for x in row['about']]

        edu_matches = [s for s in row['education'] if any(xs in s for xs in FDorUSD)]
        row['relevant_education'] = edu_matches

        skills_matches = [s for s in row['courses'] if any(xs in s for xs in SKILLS)] + [s for s in about if any(xs in s for xs in SKILLS)]
        row['relevant_skills'] = skills_matches
    except TypeError:
        pass

    return(row)

In [ ]:
drs = get_doctors()

  0%|          | 0/104 [00:00<?, ?it/s]

### Создание человеко-понятной плоской структуры (DataFrame)

In [28]:
df = pd.DataFrame(drs)
df = df.fillna('')
df['relevant_education'] = ''
df['relevant_skills'] = ''

df = df.apply(get_doctor_features, axis=1)

In [36]:
# превращение списков в текст для облегчения чтения таблицы человеком
for column in ['about', 'profile', 'education', 'courses', 'relevant_education', 'relevant_skills']:
    df[column] = df[column].apply(lambda x: "; ".join(x) if not len(x)==0 else '')

# очищение датасета от строк, в которых не нашлось нужных ключевых слов ни по одному из параметров
df = df.query('relevant_education != "" or relevant_skills != ""')

#выгрузка файла в excel
df.to_excel('neuros.xlsx', index=False)